# 交通事故データ解析

In [69]:
import cufflinks
cufflinks.go_offline()
import pandas as pd
from glob import glob

## CSVデータ読み込み

In [41]:
df = pd.concat([pd.read_csv(fn, encoding="shift-jis") for fn in glob("data/交通事故-(平成*年).csv")])

### 最初の5行

In [42]:
df.head()

,事故内容,事故類型,名称,地点経度,地点緯度,当事者種別,所轄警察署,発生場所,発生年月日,経度,緯度
0,軽傷事故,車両相互,全事故,133.476678,35.270006,普通乗用×普通乗用,黒坂署,江府町大字武庫,2012/12/12,NaN,NaN
1,軽傷事故,車両相互,全事故,133.470894,35.362625,軽乗用×普通乗用,黒坂署,伯耆町岩立,2012/11/18,NaN,NaN
2,軽傷事故,車両相互,全事故,133.402917,35.384747,自転車×軽乗用貨物,黒坂署,伯耆町大殿,2012/11/18,NaN,NaN
3,軽傷事故,車両相互,全事故,133.398069,35.387817,軽乗用×軽乗用,黒坂署,伯耆町大殿,2012/11/02,NaN,NaN
4,軽傷事故,車両相互,全事故,133.553078,35.295319,軽二輪×軽乗用貨物×二輪751ｃｃ以上,黒坂署,江府町大字助澤,2012/07/15,NaN,NaN


### 最後の5行

In [40]:
df.tail()

,事故内容,事故類型,名称,地点経度,地点緯度,当事者種別,所轄警察署,発生場所,発生年月日,経度,緯度
1350,軽傷事故,人対車両,歩行者事故,NaN,NaN,軽乗用×歩行者,米子署,米子市石井,2015/12/20,133.344970,35.404800
1351,重傷事故,人対車両,歩行者事故,NaN,NaN,普通乗用×歩行者,境港署,境港市蓮池町,2015/11/15,133.224206,35.539372
1352,重傷事故,人対車両,歩行者事故,NaN,NaN,普通乗用×歩行者,黒坂署,日野郡日野町根雨,2015/11/25,133.441388,35.242385
1353,死亡事故,人対車両,歩行者事故,NaN,NaN,軽乗用×歩行者,黒坂署,日野郡日野町本郷,2015/12/23,133.430703,35.237712
1354,重傷事故,人対車両,歩行者事故,NaN,NaN,普通乗用×歩行者,黒坂署,日野郡江府町大字大河原,2015/11/4,133.549556,35.345829


### 二つの列をマージする

CSVデータでは"地点経度"と"経度"のような重複な列がある、この２列を一つの列にする。

In [39]:
for col in ["経度", "緯度"]:
    df[col] = df[col].combine_first(df["地点" + col])
df2 = df.drop(["地点経度", "地点緯度"], axis=1)
df2.head()

,事故内容,事故類型,名称,当事者種別,所轄警察署,発生場所,発生年月日,経度,緯度
0,軽傷事故,車両相互,全事故,普通乗用×普通乗用,黒坂署,江府町大字武庫,2012/12/12,133.476678,35.270006
1,軽傷事故,車両相互,全事故,軽乗用×普通乗用,黒坂署,伯耆町岩立,2012/11/18,133.470894,35.362625
2,軽傷事故,車両相互,全事故,自転車×軽乗用貨物,黒坂署,伯耆町大殿,2012/11/18,133.402917,35.384747
3,軽傷事故,車両相互,全事故,軽乗用×軽乗用,黒坂署,伯耆町大殿,2012/11/02,133.398069,35.387817
4,軽傷事故,車両相互,全事故,軽二輪×軽乗用貨物×二輪751ｃｃ以上,黒坂署,江府町大字助澤,2012/07/15,133.553078,35.295319


同じ日付、同じ場所で発生した事故を重複入力の可能性があるので、重複したデータを削除します。

In [60]:
df2.drop_duplicates(subset=["発生年月日", "経度", "緯度"], inplace=True)

## データの統計

### 事故内容の統計

In [61]:
事故内容 = pd.value_counts(df2["事故内容"])
事故内容

軽傷事故    3529
重傷事故     751
死亡事故     106
人対車両       1
車両相互       1
Name: 事故内容, dtype: int64

"車両相互"と"人対車両"は"事故内容"ではなく、"事故類型"のはずです。データ記入のミスだと思うので、該当する行を削除する。

In [62]:
df3 = df2[df2.事故内容.isin(事故内容[:3].index)]

"当事者種別"列を"×"で分割し、当事者の度数分布図を出力します。

In [70]:
当事者 = df3["当事者種別"].str.get_dummies("×").sum().sort_values()
当事者[当事者 >= 50].iplot(kind="bar", dimensions=(800, 500))

各個"事故内容"にたいして、"当事者種別"を統計します。

In [65]:
事故原因 = df3.groupby("事故内容").apply(lambda o:o["当事者種別"].str.get_dummies("×").sum()).unstack(0).fillna(0)
事故原因 = 事故原因.loc[当事者.index]
事故原因.tail()

事故内容,死亡事故,軽傷事故,重傷事故
軽乗用貨物,8.0,344.0,65.0
歩行者,39.0,241.0,183.0
自転車,11.0,406.0,176.0
軽乗用,40.0,2004.0,338.0
普通乗用,39.0,2182.0,325.0


50件事故以上の"当事者種別"にたいして、"事故原因"の割合図を出力します。

In [66]:
事故原因[事故原因.sum(axis=1) > 50].iplot(kind="bar", dimensions=(800, 500), barmode="stack")

事故を地図上に表すため、データを整理します。

In [67]:
df_map = pd.DataFrame({
        "x":df["経度"],
        "y":df["緯度"],
        "icon":"car",
        "color":"green",
        "text":df["発生年月日"] + ": " + df["当事者種別"]
    })

s = df["当事者種別"]
for icon, pattern in zip(["truck", "motorcycle", "bicycle", "user"], ["貨物", "原付|二輪", "自転車", "歩行者"]):
    df_map.loc[s.str.contains(pattern), "icon"] = icon
    
s = df["事故内容"]
for color, pattern in zip(["green", "orange", "red"], ["軽傷", "重傷", "死亡"]):
    df_map.loc[s.str.contains(pattern), "color"] = color
    
df_map.dropna(inplace=True)
df_map.head()

,color,icon,text,x,y
0,green,car,2014/9/18: 軽乗用×軽乗用×普通乗用,133.364474,35.431586
1,orange,car,2014/10/4: 軽乗用×普通乗用,134.231846,35.456160
2,orange,user,2014/11/5: 軽乗用×歩行者,133.335616,35.423464
3,green,bicycle,2014/11/15: 軽乗用×自転車,133.348036,35.435128
4,green,bicycle,2014/11/20: 軽乗用×自転車,133.337914,35.440842


地図を出力します。

In [68]:
import folium
from fastmarker import FastMarkerCluster

fig = folium.Figure()
map_ = folium.Map(location=[df_map.y.mean(), df_map.x.mean()], zoom_start=10)

def create_marker(row):
    icon = L.AwesomeMarkers.icon({
                        icon: row.icon,
                        prefix: 'fa',
                        markerColor: row.color,
                        })    
    marker = L.marker(L.LatLng(row.y, row.x))
    marker.setIcon(icon)
    marker.bindPopup(row.text)
    marker.on('mouseover', lambda e:marker.openPopup())
    marker.on('mouseout', lambda e:marker.closePopup())
    return marker

cluster = FastMarkerCluster(df_map.to_json(orient="records"), 
                            callback=create_marker, 
                            settings={"disableClusteringAtZoom":15}).add_to(map_)
map_.add_to(fig)